# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [1]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = "Jesper Ekstrøm"
STUDENT_1_STUDENT_NUMBER = "5900468"
STUDENT_1_NETID = "jekstrom"
STUDENT_1_EMAIL = "J.Ekstrom@student.tudelft.nl"

In [2]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 2d.1 - Linearization of the double pendulum system (20p)

**Author:** Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

The purpose of this task is to derive a discrete-time, linear state-space representation (i.e. the $A_\mathrm{d}$, $B_\mathrm{d}$, $C_\mathrm{d}$, and $D_\mathrm{d}$ matrices) by linearizing the double pendulum system around an equilbrium / a trajectory.

In [3]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

from distutils.util import strtobool
from functools import partial
from IPython.display import display, HTML  # For animations in the notebook
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import debug, jacfwd, jit, lax, vmap
import jax.numpy as jnp
from jax.scipy import linalg
import os
from pathlib import Path
from typing import Callable, Dict, Tuple

from jax_double_pendulum.dynamics import (
    dynamical_matrices,
    continuous_forward_dynamics,
    continuous_inverse_dynamics,
    discrete_forward_dynamics,
    continuous_linear_state_space_representation,
)
from jax_double_pendulum.robot_parameters import ROBOT_PARAMS

# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

## Theory (6p)

### Deriving the (nonlinear) state-space representation (1p)

We usually consider closed-loop equations of motion in Euler-Lagrange form:

\begin{equation}
M(\theta) \: \ddot{\theta} + C(\theta, \dot{\theta}) \: \dot{\theta} + G(\theta) = k_\mathrm{p} \, (\theta^\mathrm{d}-\theta) + k_\mathrm{d} \, (\dot{\theta}^\mathrm{d}-\dot{\theta}) + \tau
\end{equation}

where the feedback PD controller is already included and $\tau$ is some feedforward torque.

Before linearizing, we first have to bring these closed-loop equations of motion into (nonlinear) state-space representation, which are a subset of the Ordinary Differential Equations (ODEs). In the continuous time-domain, the nonlinear state-space has for systems affine in control the following structure:
\begin{equation}
\begin{split}
\dot{x} &= f(x) + g(x) \: \tau\\
y &= h(x, \tau) 
\end{split}
\end{equation}

We want the output $y$ to be defined as the link angles $\theta$. How does the nonlinear state-space then look like?

**A:**
\begin{equation}
\begin{split}
\frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \theta\\ \dot{\theta} \end{bmatrix} &= 
\begin{bmatrix}
    \dot{\theta}\\
    M^{-1}(\theta) (C(\theta, \dot{\theta}) \: \dot{\theta} + G(\theta) - k_\mathrm{p} \: (\theta^\mathrm{d}-\theta) - k_\mathrm{d} \: (\dot{\theta}^\mathrm{d}-\dot{\theta}))
\end{bmatrix} + \begin{bmatrix}
    0^{2 \times 2}\\
    M^{-1}(\theta)
\end{bmatrix} \: \tau\\
y &= \theta
\end{split}
\end{equation}

**B:**
\begin{equation}
\begin{split}
\frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \theta\\ \dot{\theta} \end{bmatrix} &= 
\begin{bmatrix}
    \dot{\theta}\\
    M^{-1}(\theta) (-C(\theta, \dot{\theta}) \: \dot{\theta} - G(\theta) + k_\mathrm{p} \: (\theta^\mathrm{d}-\theta) + k_\mathrm{d}\: (\dot{\theta}^\mathrm{d}-\dot{\theta}))
\end{bmatrix} + \begin{bmatrix}
    0^{2 \times 2}\\
    M^{-1}(\theta)
\end{bmatrix} \: \tau\\
y &= \theta
\end{split}
\end{equation}

**C:**
\begin{equation}
\begin{split}
\frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \dot{\theta} \end{bmatrix} &= 
\begin{bmatrix}
    M^{-1}(\theta) (-C(\theta, \dot{\theta}) \: \dot{\theta} - G(\theta) + k_\mathrm{p} \: (\theta^\mathrm{d}-\theta) + k_\mathrm{d} \: (\dot{\theta}^\mathrm{d}-\dot{\theta}))
\end{bmatrix} + \begin{bmatrix}
    M^{-1}(\theta)
\end{bmatrix} \: \tau\\
y &= \int \theta
\end{split}
\end{equation}

**D:**
\begin{equation}
\begin{split}
\frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \theta\\ \dot{\theta} \end{bmatrix} &= 
\begin{bmatrix}
    \dot{\theta}\\
    M^{-1}(\theta) (-C(\theta, \dot{\theta}) \: \dot{\theta} - G(\theta))
\end{bmatrix} + \begin{bmatrix}
    0^{2 \times 2}\\
    M^{-1}(\theta)
\end{bmatrix} \: \tau\\
y &= \theta
\end{split}
\end{equation}

In [4]:
# please write the answer ("A", "B", "C", or "D") into the `answer_1` variable
# YOUR CODE HERE
answer_1 = "B"

In [5]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert answer_1 in ["A", "B", "C", "D"], 'Please answer "A", "B", "C", or "D"'


### Error notation around equilibrium (2p)

Next, we need to establish an error notation, such that we can linearize around $\delta x = 0$ and $\delta \tau = 0$:

\begin{equation}
    A = \frac{\partial f}{\partial \delta x} \big \vert_{\delta x=0, \delta \tau = 0}
    \qquad 
    B = \frac{\partial g}{\partial \delta \tau} \big \vert_{\delta x=0, \delta \tau = 0}
    \qquad 
    C = \frac{\partial h}{\partial \delta x} \big \vert_{\delta x=0, \delta \tau = 0}
    \qquad 
    D = \frac{\partial h}{\partial \delta \tau} \big \vert_{\delta x=0, \delta \tau = 0}
\end{equation}

where $A \in \mathbb{R}^{4 \times 4}$ is the state transition matrix, $B \in \mathbb{R}^{2 \times 2}$ is the input matrix, $C \in \mathbb{R}^{2 \times 4}$ is the output matrix and $D \in \mathbb{R}^{2 \times 2}$ encapsulates the feed-through terms.

The linearized system dynamics are then given by:

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \delta x &=  A \: \delta x + B \: \delta \tau\\
    y &= C \: \delta x + D \: \delta \tau
\end{split}
\end{equation}

How do $\delta x$ and $\delta \tau$ need to be defined to linearize the above defined closed-loop system around the trajectory $\theta^\mathrm{d}$, $\dot{\theta}^\mathrm{d}$, $\ddot{\theta}^\mathrm{d}$? In the following, $\bar{x}$ and $\bar{\tau}$ will signify to the equilibrium state and torque respectively.


**A:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = 0
\end{equation}

**B:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = M(\theta) \: \ddot{\theta}^\mathrm{d} + C(\theta, \dot{\theta}) \: \dot{\theta}^\mathrm{d} + G(\theta)
\end{equation}

**C:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} =M(\theta) \: \ddot{\theta} + C(\theta, \dot{\theta}) \: \dot{\theta} + G(\theta)
\end{equation}

**D:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = M(\theta^\mathrm{d}) \: \ddot{\theta}^\mathrm{d} + C(\theta^\mathrm{d}, \dot{\theta}^\mathrm{d}) \: \dot{\theta}^\mathrm{d} + G(\theta^\mathrm{d})
\end{equation}

**E:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        0\\
        0
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = 0
\end{equation}

**F:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = G(\theta^\mathrm{d})
\end{equation}

**G:**

\begin{equation}
    \delta x = \underbrace{\begin{bmatrix}
        \theta\\
        \dot{\theta}
    \end{bmatrix}}_{x} 
    - 
    \underbrace{\begin{bmatrix}
        \theta^\mathrm{d}\\
        \dot{\theta}^\mathrm{d}
    \end{bmatrix}}_{\bar{x}}
    \qquad
    \delta \tau = \tau - \bar{\tau}
    \qquad
    \bar{\tau} = M(\theta^\mathrm{d}) \: \ddot{\theta}^\mathrm{d} + C(\theta^\mathrm{d}, \dot{\theta}^\mathrm{d}) \: \dot{\theta}^\mathrm{d} + G(\theta^\mathrm{d})^\mathrm{d} - k_\mathrm{p} \, \theta^\mathrm{d} - k_\mathrm{d} \, \dot{\theta}^\mathrm{d}
\end{equation}


In [6]:
# please write the answer ("A", "B", "C", "D", "E", "F", "G") into the `answer_2` variable
# YOUR CODE HERE
answer_2 = "D"

In [7]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert answer_2 in [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
], 'Please answer "A", "B", "C", "D", "E", "F", or "G"'


### State-space representation in error notation (2p)

Please plug-in the error notation $\delta x$, $\delta \tau$ into the nonlinear, closed-loop state-space representation and simplify. How do the nonlinear error dynamics look like? **Note:** if not specified differently, all dynamical matrices (e.g. $M$, $C$, and $G$) are evaluated at $\theta = \theta^\mathrm{d} + \delta \theta$, $\dot{\theta} = \dot{\theta}^\mathrm{d} + \delta \dot{\theta}$.

**A:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
    \delta \dot{\theta}\\
    M^{-1} \: (- C \: \delta \dot{\theta} + k_\mathrm{p} \: \delta \theta + k_\mathrm{d} \: \delta \dot{\theta})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**B:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
    \delta \dot{\theta}\\
    M^{-1} \: (- C \: \delta \dot{\theta} - k_\mathrm{p} \: \delta \theta - k_\mathrm{d} \: \delta \dot{\theta})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**C:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
    \delta \dot{\theta}\\
    -\ddot{\theta}^\mathrm{d} + M^{-1} \: (- C \: (\dot{\theta}^\mathrm{d} + \delta \dot{\theta}) - G - k_\mathrm{p} \: \delta \theta - k_\mathrm{d} \: \delta \dot{\theta} + \bar{\tau})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**D:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
   \dot{\theta} + \delta \dot{\theta}\\
    -\ddot{\theta}^\mathrm{d} + M^{-1} \: (- C \: (\dot{\theta}^\mathrm{d} + \delta \dot{\theta}) - G - k_\mathrm{p} \: \delta \theta - k_\mathrm{d} \: \delta \dot{\theta} + \bar{\tau})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**E:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
   \dot{\theta} + \delta \dot{\theta}\\
    -\ddot{\theta}^\mathrm{d} + M^{-1} \: (- C \: (\dot{\theta}^\mathrm{d} + \delta \dot{\theta}) - G - k_\mathrm{p} \: \delta \theta - k_\mathrm{d} \: \delta \dot{\theta})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**F:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
    \delta \dot{\theta}\\
    \ddot{\theta}^\mathrm{d} + M^{-1} \: (- C \: (\dot{\theta}^\mathrm{d} + \delta \dot{\theta}) - G - k_\mathrm{p} \: \delta \theta - k_\mathrm{d} \: \delta \dot{\theta} + \bar{\tau})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

**G:**

\begin{equation}
\begin{split}
    \frac{\mathrm{d}}{\mathrm{d}t} \begin{bmatrix} \delta \theta\\ \delta \dot{\theta} \end{bmatrix}
    &= \begin{bmatrix}
    \delta \dot{\theta}\\
    M^{-1} \: (- C \: (\dot{\theta}^\mathrm{d} + \delta \dot{\theta}) - G + \bar{\tau})
    \end{bmatrix} + \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix} \: \delta \tau\\
    y &= \theta^\mathrm{d} + \delta \theta
\end{split}
\end{equation}

In [8]:
# please write the answer ("A", "B", "C", "D", "E", "F", "G") into the `answer_3` variable
# YOUR CODE HERE
answer_3 = "C"

In [9]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert answer_3 in [
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
], 'Please answer "A", "B", "C", "D", "E", "F", or "G"'


### Deriving the linear state-space matrices (1p)

After completing the previous questions, we now have the dynamics in a structure where we can linearize using first-order Taylor expansion and derive the linear state-space representation. Unfortunately, the analytical calculation of $A$ is slightly involved and therefore out-of-scope for this assignment. Therefore, in the remainder of this task we will use JAX autograd to help us with the numerical computation of the state-space system. Still, we want to analytically derive $B$, $C$, and $D$ to help us verify our _autograd_ implementation later. Please choose the answer which corresponds to the correct derivation of the state-space matrices.

**A:**

\begin{equation}
    B = \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix}
    \qquad
    C = \begin{bmatrix}
        \mathbb{I}_{2} & 0^{2 \times 2}
    \end{bmatrix}
    \qquad
    D = 0^{2 \times 2}
\end{equation}

**B:**

\begin{equation}
    B = \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1}
    \end{bmatrix}
    \qquad
    C = \begin{bmatrix}
        0^{2 \times 2} & \mathbb{I}_{2}
    \end{bmatrix}
    \qquad
    D = 0^{2 \times 2}
\end{equation}

**C:**
\begin{equation}
    B = \begin{bmatrix}
        M^{-1}\\
        0^{2 \times 2}\\
    \end{bmatrix}
    \qquad
    C = \begin{bmatrix}
        \mathbb{I}_{2} & 0^{2 \times 2}
    \end{bmatrix}
    \qquad
    D = 0^{2 \times 2}
\end{equation}

**D:**

\begin{equation}
    B = \begin{bmatrix}
        0^{2 \times 2}\\
        M^{-1} \bar{\tau}
    \end{bmatrix}
    \qquad
    C = \begin{bmatrix}
        0^{2 \times 2} & \mathbb{I}_{2}
    \end{bmatrix}
    \qquad
    D = 0^{2 \times 2}
\end{equation}

where $\mathbb{I}$ refers to the identity matrix.

In [10]:
# please write the answer ("A", "B", "C", or "D") into the `answer_4` variable
# YOUR CODE HERE
answer_4 = "A"

In [11]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert answer_4 in ["A", "B", "C", "D"], 'Please answer "A", "B", "C", or "D"'


## Linearization of system using JAX autograd (3p)

The goal of this subtask is to linearize the system around the given equilibrium link angles $\bar{\theta}$, equilibrium link angular velocities $\bar{\dot{\theta}}$, and torque $\bar{\tau}$. For this, we rely on JAX autograd to gather the gradients of the state time derivatives / outputs with respect to the states / inputs. Please implement the `continuous_state_space_dynamics` and `continuous_linear_state_space_representation_autograd` functions in the `linearization` notebook. Below, we provide you with some hints about the necessary steps:

1. Implement the (nonlinear) state-space dynamics $\dot{x} = f(x, \tau)$, $y = g(x, \tau)$ into the `continuous_state_space_dynamics` function. For this, we define the state as $x = (\theta^\mathrm{T}, \dot{\theta}^\mathrm{T})^\mathrm{T}$ and the output as $y = \theta$. You can make use of the provided `continuous_forward_dynamics_fn` function, which maps the system state and the applied torque to the link angular accelerations: $(\theta, \dot{\theta}, \tau) \mapsto \ddot{\theta}$.

2. Then within `continuous_linear_state_space_representation_autograd`, use [`jax.jacfwd`](https://jax.readthedocs.io/en/latest/_autosummary/jax.jacfwd.html) to get the gradients of $\dot{x}$ and $y$ with respect to $x$ and $\tau$ and return the linear state-space representation. 


**Hint:** We assume a function signature `my_fn(x: jnp.Array, y: jnp.Array, z: jnp.Array) -> a: jnp.Array, b: jnp.Array`. Now, we apply [`jax.jacfwd`](https://jax.readthedocs.io/en/latest/_autosummary/jax.jacfwd.html) to this function with the setting `argnums=(0, 1)` and assign it to `my_fn_jacobian`. Calling this Jacobian function wit the input `(x, y, z)`, this Jacobian function would provide the output `(da_dx: jnp.Array, da_dy: jnp.Array, da_dz: jnp.Array), (db_dx: jnp.Array, db_dy: jnp.Array, db_dz: jnp.Array)`, where `da_dx` corresponds to $\frac{\partial a}{\partial x}$, `db_dx` corresponds to $\frac{\partial b}{\partial x}$ etc.


In [12]:
# import continuous_linear_state_space_representation_autograd from linearization.ipynb
from ipynb.fs.full.linearization import (
    continuous_linear_state_space_representation_autograd,
)


# make sure that continuous_forward_dynamics_fn has the correct signature
_continuous_forward_dynamics_fn = partial(continuous_forward_dynamics, ROBOT_PARAMS)

# define equilibrium states and inputs
_th_eq = jnp.pi / 2 * jnp.ones((2,))
_th_d_eq = jnp.zeros((2,))
_tau_eq = jnp.zeros((2,))

# linearize using autograd
_A_ag, _B_ag, _C_ag, _D_ag = continuous_linear_state_space_representation_autograd(
    _continuous_forward_dynamics_fn,
    _th_eq,  # th_eq
    _th_d_eq,  # th_d_eq
    _tau_eq,  # tau_eq
)

print("Autograd results")
print("A = \n", _A_ag)
print("B = \n", _B_ag)
print("C = \n", _C_ag)
print("D = \n", _D_ag)

# we perform the same linearization using the analytical solution (derived using symbolic math)
from jax_double_pendulum.dynamics import continuous_linear_state_space_representation

_A_al, _B_al, _C_al, _D_al = continuous_linear_state_space_representation(
    ROBOT_PARAMS, _th_eq, _th_d_eq, _tau_eq
)

print("Analytical results")
print("A = \n", _A_al)
print("B = \n", _B_al)
print("C = \n", _C_al)
print("D = \n", _D_al)

Autograd results
A = 
 [[  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]
 [  8.07882353  -1.88855615   0.           0.        ]
 [-13.84941176  11.64609626   0.           0.        ]]
B = 
 [[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.03743316 -0.06417112]
 [-0.06417112  0.39572193]]
C = 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
D = 
 [[0. 0.]
 [0. 0.]]
Analytical results
A = 
 [[  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]
 [  8.07882353  -1.88855615  -0.          -0.        ]
 [-13.84941176  11.64609626   0.           0.        ]]
B = 
 [[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.03743316 -0.06417112]
 [-0.06417112  0.39572193]]
C = 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
D = 
 [[0. 0.]
 [0. 0.]]


In [13]:
# DO NOT REMOVE OR MODIFY THIS CELL

# we compare the autograd and an analytical solution and make sure that they are the same
assert jnp.allclose(_A_ag, _A_al)
assert jnp.allclose(_B_ag, _B_al)
assert jnp.allclose(_C_ag, _C_al)
assert jnp.allclose(_D_ag, _D_al)


## Discretization (2p)

In the previous sections, we had derived the state-space description in the continuous time domain. As Iterative Learning Control (ILC) operates in the discrete time domain, we now need to discretize the state space description. Please implement below the zero-order hold method to derive the discrete-time state-space description. The discrete-time, linearized dynamics are then given by:

\begin{equation}
\begin{split}
x_{k+1} = A_\mathrm{d} \, x_k + B_\mathrm{d} \, u_k\\
y_{k} = C_\mathrm{d} \, x_k + D_\mathrm{d} \, u_k
\end{split}
\end{equation}

Please implement the `cont2discrete_zoh` function into the `linearization` noteboook.

In [14]:
# import cont2discrete_zoh from linearization.ipynb
from ipynb.fs.full.linearization import cont2discrete_zoh

_A = jnp.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0.5, 0, 0], [0.5, 0, 0, 0]])
_B = jnp.array([[0, 0], [0, 0], [1, 0], [0, 1]])
_C, _D = jnp.zeros((2, 4)), jnp.zeros((2, 2))
_Ad, _Bd, _Cd, _Dd = cont2discrete_zoh(0.01, _A, _B, _C, _D)

print("Computed discrete-time state-space representation:")
print("Ad\n", _Ad)
print("Bd\n", _Bd)
print("Cd\n", _Cd)
print("Dd\n", _Dd)

Computed discrete-time state-space representation:
Ad
 [[1.00000000e+00 2.50000000e-05 1.00000000e-02 8.33333333e-08]
 [2.50000000e-05 1.00000000e+00 8.33333333e-08 1.00000000e-02]
 [4.16666667e-08 5.00000000e-03 1.00000000e+00 2.50000000e-05]
 [5.00000000e-03 4.16666667e-08 2.50000000e-05 1.00000000e+00]]
Bd
 [[5.00000000e-05 2.08333333e-10]
 [2.08333333e-10 5.00000000e-05]
 [1.00000000e-02 8.33333333e-08]
 [8.33333333e-08 1.00000000e-02]]
Cd
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Dd
 [[0. 0.]
 [0. 0.]]


In [15]:
# DO NOT REMOVE OR MODIFY THIS CELL

# check output dimensions
assert _Ad.shape == (4, 4), "Ad has wrong shape"
assert _Bd.shape == (4, 2), "Bd has wrong shape"
assert _Cd.shape == (2, 4), "Cd has wrong shape"
assert _Dd.shape == (2, 2), "Dd has wrong shape"

# compare your output with the output of scipy.signal.cont2discrete
from scipy.signal import cont2discrete as scipy_cont2discrete

(_Ad_target, _Bd_target, _Cd_target, _Dd_target, _dt) = scipy_cont2discrete(
    (_A, _B, _C, _D), 0.01
)


print("Scipy discrete-time state-space representation:")
print("Ad\n", _Ad_target)
print("Bd\n", _Bd_target)
print("Cd\n", _Cd_target)
print("Dd\n", _Dd_target)

grader_points = 0

if jnp.allclose(_Ad, _Ad_target):
    grader_points += 0.5

if jnp.allclose(_Bd, _Bd_target):
    grader_points += 0.5

if jnp.allclose(_Cd, _Cd_target):
    grader_points += 0.5

if jnp.allclose(_Dd, _Dd_target):
    grader_points += 0.5

grader_points

Scipy discrete-time state-space representation:
Ad
 [[1.00000000e+00 2.50000000e-05 1.00000000e-02 8.33333333e-08]
 [2.50000000e-05 1.00000000e+00 8.33333333e-08 1.00000000e-02]
 [4.16666667e-08 5.00000000e-03 1.00000000e+00 2.50000000e-05]
 [5.00000000e-03 4.16666667e-08 2.50000000e-05 1.00000000e+00]]
Bd
 [[5.00000000e-05 2.08333333e-10]
 [2.08333333e-10 5.00000000e-05]
 [1.00000000e-02 8.33333333e-08]
 [8.33333333e-08 1.00000000e-02]]
Cd
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Dd
 [[0. 0.]
 [0. 0.]]


2.0

## Verification of linearization on an open-loop system (5p)

Now, we want verify the correctness of the linearization. For this, we linearize the open-loop system (i.e. **without** feedback controller) around the equilibrium link angles $\bar{\theta} = (0, 0)^\mathrm{T}$, link angular velocities $\bar{\dot{\theta}} = (0, 0)^\mathrm{T}$, the link angular accelerations $\bar{\ddot{\theta}} = (0, 0)^\mathrm{T}$ and the equilibrium torque $\bar{\tau}$. This will result in a **time-invariant** linear system. Please note that in the next exercise, we will extend this to the case of a time-varying linear system and will then linearize the closed-loop system (i.e. including the PD feedback controller).

In the first step, we linearize and discretize the system to get the discrete-time, linear state-space representation. In the cell below, please implement the following parts:
1. First, compute the equilibrium torque using the equations of motion of the system.
2. Then, linearize the system around $\bar{\theta}$, $\bar{\dot{\theta}}$, and $\bar{\tau}$.
3. Next, discretize the resulting state-space representation to get $A_\mathrm{d} \in \mathbb{R}^{4 \times 4}$, $B_\mathrm{d} \in \mathbb{R}^{4 \times 2}$m $C_\mathrm{d} \in \mathbb{R}^{2 \times 4}$, $D_\mathrm{d} \in \mathbb{R}^{2 \times 2}$.


In [16]:
# time-step
sim_dt = 1e-2  # [s]

# equilibrium settings for link angles and link angular velocities
th_eq = jnp.zeros((2,))
th_d_eq = jnp.zeros((2,))

# First, compute the equilibrium torque tau_eq using the Equations of Motion
# Hint: make use of the dynamical_matrices function in the `jax_double_pendulum.dynamics` module
tau_eq = jnp.zeros((2,))
# YOUR CODE HERE
M, C, G = dynamical_matrices(ROBOT_PARAMS, th_eq, th_d_eq)
tau_eq = G
assert tau_eq.shape == (2, )

print("Computed equilibrium torque as", tau_eq, "Nm")

# Then, linearize the system and get the discrete-time state-space description
Ad, Bd = jnp.zeros((4, 4)), jnp.zeros((4, 2))
Cd, Dd = jnp.zeros((2, 4)), jnp.zeros((2, 2))

# YOUR CODE HERE
A, B, C, D = continuous_linear_state_space_representation_autograd(_continuous_forward_dynamics_fn, th_eq, th_d_eq, tau_eq,)
Ad, Bd, Cd, Dd = cont2discrete_zoh(sim_dt, A, B, C, D)

print("Computed discrete-time state-space representation:")
print("Ad\n", Ad)
print("Bd\n", Bd)
print("Cd\n", Cd)
print("Dd\n", Dd)

Computed equilibrium torque as [215.82  29.43] Nm
Computed discrete-time state-space representation:
Ad
 [[1.   0.   0.01 0.  ]
 [0.   1.   0.   0.01]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]
Bd
 [[ 1.87165775e-06 -3.20855615e-06]
 [-3.20855615e-06  1.97860963e-05]
 [ 3.74331551e-04 -6.41711230e-04]
 [-6.41711230e-04  3.95721925e-03]]
Cd
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Dd
 [[0. 0.]
 [0. 0.]]


In [17]:
# DO NOT REMOVE OR MODIFY THIS CELL


Please implement `linearized_discrete_forward_dynamics` in the `linearization.ipynb` notebook. It should use the discrete, linear state-space representation derived above to compute the discrete dynamics of the system. 

**Hint:** Use finite differences to compute the link angular acceleration $\ddot{\theta}$.

In [18]:
# import linearized_discrete_forward_dynamics from linearization.ipynb
from ipynb.fs.full.linearization import linearized_discrete_forward_dynamics

_dt = 1e-2  # s
_th = jnp.zeros((2,))
_th_d = jnp.array([7.0, 2.0])
_tau = jnp.array([1.0, 3.0])

# compute one timestep using `linearized_discrete_forward_dynamics`
_th_next, _th_d_next, _th_dd = linearized_discrete_forward_dynamics(
    Ad,
    Bd,
    Cd,
    Dd,
    th_eq,
    th_d_eq,
    tau_eq,
    dt=_dt,
    th=_th,
    th_d=_th_d,
    tau=_tau,
)

print("th_next\n", _th_next)
print("th_d_next\n", _th_d_next)
print("th_dd\n", _th_dd)

# make sure that your implementation is correct
_th_next_target = jnp.array([0.06968273, 0.02016632])
_th_d_next_target = jnp.array([6.93654652, 2.0332631])
_th_dd_target = jnp.array([-6.34534759, 3.32631016])
assert jnp.allclose(_th_next_target, _th_next)
assert jnp.allclose(_th_d_next_target, _th_d_next)
assert jnp.allclose(_th_dd_target, _th_dd)


th_next
 [0.06968273 0.02016632]
th_d_next
 [6.93654652 2.0332631 ]
th_dd
 [-6.34534759  3.32631016]


Then, we roll-out the discrete dynamics of both the linearized and the nonlinear system over a duration of 4s. We do not apply any external torque to the system and rely on the (linearized) gravity forces to move the double pendulum. **Hint:** After adjusting the signature of `linearized_discrete_forward_dynamics` using `partial`, pass it to the `simulate_robot` method to rollout the linearized system. Please investigate for how long the linearized description of the dynamics is decently accurate.

In [19]:
# define time steps
sim_duration = 4.0  # [s]
t_ts = sim_dt * jnp.arange(int(sim_duration / sim_dt))

from jax_double_pendulum.analysis import *
from jax_double_pendulum.robot_simulation import simulate_robot
from jax_double_pendulum.visualization import animate_robot

# simulate the nominal dynamics
sim_ts = simulate_robot(rp=ROBOT_PARAMS, t_ts=t_ts, th_0=th_eq, th_d_0=th_d_eq)

# simulate the linearized system from the same initial conditions
# Hint: you can pass the linear, discrete forward dynamics as an argument with the keyword
# `discrete_forward_dynamics_fn` to the `jax_double_pendulum.robot_simulation.simulate_robot` function
# YOUR CODE HERE
linearized_discrete_forward_dynamics_fn = partial(linearized_discrete_forward_dynamics, Ad, Bd, Cd, Dd, th_eq, th_d_eq, tau_eq)
linearized_sim_ts = simulate_robot(rp=ROBOT_PARAMS, t_ts=t_ts, th_0=th_eq, th_d_0=th_d_eq, 
                                    discrete_forward_dynamics_fn=linearized_discrete_forward_dynamics_fn) 

# evaluate the control performance quantitatively
rmse_th, rmse_th_d, rmse_th_dd = compute_configuration_space_rmse(
    sim_ts, linearized_sim_ts
)
rmse_x, rmse_x_d, rmse_x_dd = compute_operational_space_rmse(sim_ts, linearized_sim_ts)
with jnp.printoptions(precision=3):
    print(
        "RMSE theta:",
        rmse_th,
        "rad, RMSE theta_d:",
        rmse_th_d,
        "rad/s, RMSE theta_dd:",
        rmse_th_dd,
        "rad/s^2",
    )
    print(
        "RMSE x:",
        f"{jnp.linalg.norm(rmse_x):.4f}",
        "m, RMSE x_d:",
        f"{jnp.linalg.norm(rmse_x_d):.3f}",
        "m/s, RMSE x_dd:",
        f"{jnp.linalg.norm(rmse_x_dd):.2f}",
        "m/s^2",
    )

RMSE theta: [1.642 1.869] rad, RMSE theta_d: [14.593  5.772] rad/s, RMSE theta_dd: [ 7.253 12.347] rad/s^2
RMSE x: 3.2011 m, RMSE x_d: 29.769 m/s, RMSE x_dd: 547.27 m/s^2


In [20]:
# DO NOT REMOVE OR MODIFY THIS CELL


In [21]:
# define folder where to save animations and plots
outputs_dir = Path("outputs")
outputs_dir.mkdir(parents=True, exist_ok=True)

# animate the evolution of both the nominal dynamics and the linearized dynamics
if not AUTOGRADING:
    ani = animate_robot(
        ROBOT_PARAMS,
        sim_ts=sim_ts,
        sim_hat_ts=linearized_sim_ts,
        step_skip=5,
        show=False,
        filepath=str(outputs_dir / "task_2d-1_nominal_vs_linearized_system.mp4"),
    )
    display(HTML(ani.to_html5_video()))

## Linearization of a closed-loop system about a trajectory (4p)

In the previous section, we linearized about **one** equilibrium point. However, often we want our system to follow a given trajectory parametrized by $\theta^\mathrm{d}$, $\dot{\theta}^\mathrm{d}$, and $\ddot{\theta}^\mathrm{d}$. We perform the linearization along the trajectory, which will result in a time-varying linear system: $A(k)$, $B(k)$, $C(k)$, $D(k)$. Furthermore, we now don't just consider an open-loop system anymore (i.e. without feedback controller), but now linearize the closed-loop system for which a PD controller regulates the double pendulum to follow a given trajectory.

In a first step, we need to redefine our continuous dynamics and include a PD feedback controller into dynamical description. Please implement the `closed_loop_fb_continuous_forward_dynamics` function in the `linearization` notebook. The feedback control torque should be added to the input torque `tau_ext`.

In [22]:
# import closed_loop_fb_continuous_forward_dynamics from linearization
from ipynb.fs.full.linearization import closed_loop_fb_continuous_forward_dynamics

_th = jnp.pi / 2 * jnp.ones((2,))
_th_d = jnp.ones((2,))
_tau_ext = jnp.zeros((2,))
_th_des = jnp.zeros((2,))
_th_d_des = jnp.zeros((2,))
_kp_fb = jnp.eye(2)
_kd_fb = 0.1 * jnp.eye(2)

_th_dd_cl = closed_loop_fb_continuous_forward_dynamics(
    rp=ROBOT_PARAMS,
    th=_th,
    th_d=_th_d,
    tau_ext=_tau_ext,
    th_des=_th_des,
    th_d_des=_th_d_des,
    kp_fb=_kp_fb,
    kd_fb=_kd_fb,
)

print("Computed link angular acceleration:", _th_dd_cl)

Computed link angular acceleration: [ 0.0446737  -0.55395386]


In [23]:
# DO NOT REMOVE OR MODIFY THIS CELL
assert _th_dd_cl.shape == (2,)


Subsequently, please implement the `linearize_closed_loop_fb_system_about_trajectory` function in the `linearization` notebook. Again, we provide some hints about the necessary steps:

1. Use the analytical `continuous_inverse_dynamics` of the system from the `jax_double_pendulum.dynamics` module to compute the equilbrium torque necessary to track the trajectory. You can use [`jax.vmap`](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html) to do this at every time-step.
2. Linearize the system at every time-step of the trajectory. Use [`jax.vmap`](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html) to iterate over the time-steps and `continuous_linear_state_space_representation_autograd` in conjunction with `closed_loop_fb_continuous_forward_dynamics` to linearize the closed-loop system at each time-step.
3. Discretize the time-varying state-space descriptions using the zero-order hold method. Again, you can apply [`jax.vmap`](https://jax.readthedocs.io/en/latest/_autosummary/jax.vmap.html).

In [24]:
# import linearize_closed_loop_fb_system_about_trajectory from linearization
from ipynb.fs.full.linearization import linearize_closed_loop_fb_system_about_trajectory

from jax_double_pendulum.motion_planning import (
    generate_ellipse_trajectory,
    ELLIPSE_PARAMS,
)

# generate trajectory
_traj_ts = generate_ellipse_trajectory(
    rp=ROBOT_PARAMS,
    t_ts=t_ts,
    **ELLIPSE_PARAMS,
)

# linearize and discretize the closed-loop system about the trajectory
(
    _tau_eq_ts,
    _Ad_ts,
    _Bd_ts,
    _Cd_ts,
    _Dd_ts,
) = linearize_closed_loop_fb_system_about_trajectory(
    ROBOT_PARAMS,
    _traj_ts,
    kp_fb=500 * jnp.eye(2),
    kd_fb=50 * jnp.eye(2),
)

print("Shape of linear, time-varying state-space system matrices")
print(
    "Ad_ts",
    _Ad_ts.shape,
    "Bd_ts",
    _Bd_ts.shape,
    "Cd_ts",
    _Cd_ts.shape,
    "Dd_ts",
    _Dd_ts.shape,
)
print("Linear state-space representation at first time-step of trajectory")
print("Ad at t=0s\n", _Ad_ts[0])
print("Bd at t=0s\n", _Bd_ts[0])
print("Cd at t=0s\n", _Cd_ts[0])
print("Dd at t=0s\n", _Dd_ts[0])

Shape of linear, time-varying state-space system matrices
Ad_ts (400, 4, 4) Bd_ts (400, 4, 2) Cd_ts (400, 2, 4) Dd_ts (400, 2, 2)
Linear state-space representation at first time-step of trajectory
Ad at t=0s
 [[ 9.99586826e-01  1.19621838e-04  9.92951744e-03 -5.29086274e-08]
 [ 9.68367815e-05  9.93038919e-01  1.20391210e-04  9.29594053e-03]
 [-8.24382042e-02  2.38603647e-02  9.85799079e-01  2.87155315e-05]
 [ 1.85527467e-02 -1.35808307e+00  2.34650846e-02  8.60251985e-01]]
Bd at t=0s
 [[ 1.34810349e-06 -2.09524586e-07]
 [-1.89443174e-07  1.36421167e-05]
 [ 2.68979328e-04 -4.17913323e-05]
 [-3.58492923e-05  2.66155090e-03]]
Cd at t=0s
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Dd at t=0s
 [[0. 0.]
 [0. 0.]]


In [25]:
# DO NOT REMOVE OR MODIFY THIS CELL

# define below the "correct" discretized state-space representation you should receive for the timestep t=0.
_Ad_0_target = jnp.array(
    [
        [9.99586826e-01, 1.19621838e-04, 9.92951744e-03, -5.29086274e-08],
        [9.68367815e-05, 9.93038919e-01, 1.20391210e-04, 9.29594053e-03],
        [-8.24382042e-02, 2.38603647e-02, 9.85799079e-01, 2.87155315e-05],
        [1.85527467e-02, -1.35808307e00, 2.34650846e-02, 8.60251985e-01],
    ]
)
_Bd_0_target = jnp.array(
    [
        [1.34810349e-06, -2.09524586e-07],
        [-1.89443174e-07, 1.36421167e-05],
        [2.68979328e-04, -4.17913323e-05],
        [-3.58492923e-05, 2.66155090e-03],
    ]
)
_Cd_0_target = jnp.array(
    [
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
    ]
)
_Dd_0_target = jnp.zeros((2, 2))

# assert that your solution matches this target state-space description at timestep t=0.
assert jnp.allclose(_Ad_0_target, _Ad_ts[0])
assert jnp.allclose(_Bd_0_target, _Bd_ts[0])
assert jnp.allclose(_Cd_0_target, _Cd_ts[0])
assert jnp.allclose(_Dd_0_target, _Dd_ts[0])
